In [1]:
import os
# os.environ["SDL_VIDEODRIVER"] = "x11"
# os.environ["SDL_RENDER_DRIVER"] = "software"
import pygame
pygame.init()

from cellitaire.environment.agents.ppo_agent.agent import Agent
from cellitaire.environment.cellitaire_env import CellitaireEnv
from cellitaire.environment.rewards.reward import *
from cellitaire.environment.rewards.foundation_rewards import *

board_rows = 7
board_cols = 12
num_reserved = 6
test_reward = CombinedReward([
    WinReward(weight=100, rows=board_rows, cols=board_cols),
    ScalingPlacedCardInFoundationReward(weight=1, rows=board_rows, cols=board_cols),
])
checkpoint_dir = 'tmp/experiment_4'

<frozen importlib._bootstrap>:488: RuntimeWarning: Your system is avx2 capable but pygame was not built with support for it. The performance of some of your blits could be adversely affected. Consider enabling compile time detection with environment variables like PYGAME_DETECT_AVX2=1 if you are compiling without cross compilation.


pygame 2.6.1 (SDL 2.32.56, Python 3.13.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
import time
import numpy as np

best_score = -1000
score_history = []
max_score = 0
episodes_without_best = 0
try:

    demo_env = CellitaireEnv(reward = test_reward, config_dir = checkpoint_dir, render_mode='human', frame_rate=0.1)
    demo_env.render()

    n_actions = demo_env.action_space.n,
    input_dims = (board_rows * board_cols * 4 + 53 + 4,)

    demo_agent = Agent(
        n_actions = n_actions,
        input_dims = input_dims,
        config_dir=checkpoint_dir
    )
    demo_agent.load_models()
    print('agent_loaded')
    learn_iters = 0
    avg_score = 0
    n_steps = 0
    i = 0
    while True:
        print('outer loop')
        demo_agent.load_models()
        demo_env.reset()
        observation = demo_env.get_state()
        done = False
        truncated = False
        score = 0
        while not done:
            print('inner loop')
            action, _, _ = demo_agent.choose_legal_action(observation, demo_env.get_legal_actions_as_int())
            observation_, reward, done, truncated, info = demo_env.step(action)
            n_steps += 1
            score += reward
            observation = observation_
            time.sleep(0.1)
        i += 1
        score_history.append(score)
        avg_score = np.mean(score_history[-100:])
        
        max_score = max(max_score, score)
        
        if avg_score > best_score:
            best_score = avg_score
            recent_std = np.std(score_history[-100:])
            #agent.save_models()
            episodes_without_best = 0
            print(f'episode {i:>5} | score {score:>6.1f} | avg {avg_score:>6.1f} | std {recent_std:>6.2f} | max score {max_score:>5.1f} | learning steps {learn_iters:>5} | done {done} *')
        else:
            episodes_without_best += 1
            
        if episodes_without_best % 100 == 0 and episodes_without_best > 0:
            recent_std = np.std(score_history[-100:])
            print(f'episode {i:>5} | score {score:>6.1f} | avg {avg_score:>6.1f} | std {recent_std:>6.2f} | max score {max_score:>5.1f} | learning steps {learn_iters:>5} | done {done}')
        time.sleep(8)
        print(score)
except:
    demo_env.close()

agent_loaded
outer loop
inner loop
inner loop
inner loop
inner loop
inner loop
inner loop
inner loop
inner loop
inner loop
inner loop
inner loop
inner loop
inner loop
inner loop
inner loop
inner loop
inner loop
inner loop
inner loop
inner loop
inner loop
inner loop
inner loop
inner loop
inner loop
inner loop
inner loop
inner loop
inner loop
inner loop
inner loop
inner loop
inner loop
inner loop
inner loop


In [ ]:
demo_env.close()

In [1]:
import os, threading, time, pygame

# pick a driver if you like:
# os.environ["SDL_VIDEODRIVER"] = "x11"
# os.environ["SDL_RENDER_DRIVER"] = "software"

def run_pygame():
    pygame.init()
    screen = pygame.display.set_mode((400, 300))
    pygame.display.set_caption("Pygame Test")
    clock = pygame.time.Clock()
    running = True
    t0 = time.time()
    while running and time.time() - t0 < 5:  # auto-exit after 5s
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
        screen.fill((200, 50, 50))  # red
        pygame.display.flip()
        clock.tick(30)
    pygame.quit()

print("Main thread test...")
run_pygame()  # should show a red window for ~5s

print("Threaded test...")
t = threading.Thread(target=run_pygame)
t.start()
t.join()
print("Done")


<frozen importlib._bootstrap>:488: RuntimeWarning: Your system is avx2 capable but pygame was not built with support for it. The performance of some of your blits could be adversely affected. Consider enabling compile time detection with environment variables like PYGAME_DETECT_AVX2=1 if you are compiling without cross compilation.


pygame 2.6.1 (SDL 2.32.56, Python 3.13.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
Main thread test...
Threaded test...
Done
